In [3]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [8]:
# input the data and pretreatment
data = pd.read_csv('grouped.csv')
data.drop(columns=['Unnamed: 0'])
col=['DATE']
col.extend(data.ID.unique().tolist())
result=pd.DataFrame(columns=col)
for i in data.ID.unique().tolist():
    tmp = data.loc[data['ID']==i]
    _list=[]
    for j in data.DATE.unique().tolist():
        a=tmp.loc[tmp['DATE']==j]['AMOUNT'].to_list()
        if len(a)==0:
            a.append(np.NAN)
        _list.extend(a)
    result[i]=_list

In [9]:
# delete the columns 'DATE'
result = result.iloc[:,1:]
#result

In [10]:
# delete the columns which include the 'Nan'
result_droped = result.dropna(axis=1,how='any')
dataset = result_droped
dataset.info()
dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Columns: 831 entries, 440317 to 879769
dtypes: int64(831)
memory usage: 240.3 KB


,440317,440364,440370,440384,440405,441532,442135,442137,442147,442149,...,845164,850397,858287,865898,867087,867242,873187,878235,878712,879769
0,776,18,37,107,1294,400,6,3,115,131,...,374,53,7,19,65,15,56,22,43,28
1,752,26,93,188,797,671,47,112,581,235,...,1092,112,6,43,47,11,48,53,57,30
2,208,37,143,108,235,178,20,81,307,37,...,605,89,4,25,22,50,19,31,58,8
3,1880,35,116,99,730,436,42,16,231,224,...,125,36,4,12,31,19,10,14,42,21
4,1660,17,162,249,1002,492,112,20,480,357,...,1037,86,8,61,31,73,44,64,52,20
5,2026,9,404,216,1432,962,41,65,1062,194,...,1312,34,14,79,36,18,35,46,133,58
6,1876,34,94,51,374,322,79,28,213,199,...,572,75,2,56,51,48,36,51,48,20
7,704,10,85,62,659,270,46,35,655,163,...,774,53,2,69,18,53,10,26,47,27
8,638,23,57,135,302,217,25,20,161,58,...,509,130,3,28,39,27,18,18,24,8
9,375,5,3,62,336,240,29,9,62,52,...,199,6,3,15,21,16,5,20,18,9


In [147]:
# X is the number of passengers at a given time (t) and Y is the number of passengers at the net time (t+3) // windows = 3 (look_back)

# convert an array of values into a dataset matrix
def create_dataset_X(dataset, look_back = 3):
    dataX = []
    for i in range(len(dataset)-look_back-1):
        a = dataset.iloc[i:(i+look_back)]
        dataX.append(a)
        #dataY.append(dataset[i + look_back, :])
    return dataX

def create_dataset_Y(dataset, look_back = 3):
    dataY = []
    for i in range(len(dataset)-look_back-1):
        b = dataset.iloc[(i+look_back):]
        #dataX.append(a)
        dataY.append(b)
    return dataX

In [148]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)

# split into train and test sets
train_size = int(len(dataset) * 0.85)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:],dataset[train_size:,:]
train, test = pd.DataFrame(train), pd.DataFrame(test)

,0,1,2,3,4,5,6,7,8,9,...,821,822,823,824,825,826,827,828,829,830
0,0.409490,0.756757,0.314050,0.242678,0.175824,0.206140,0.250000,0.594203,0.131352,0.212042,...,0.535783,0.235294,0.363636,0.946809,0.634921,1.000000,0.210526,0.436893,0.338462,0.181818
1,0.431986,0.635135,0.150826,0.338912,0.337767,0.319079,0.584906,0.362319,0.280920,0.607330,...,0.582205,0.373950,0.272727,0.308511,0.571429,0.156522,0.000000,0.543689,0.330769,0.068182
2,0.318805,0.148649,0.059917,0.476987,0.149219,0.393640,0.226415,0.036232,0.060403,0.070681,...,0.321470,0.117647,0.636364,0.829787,0.603175,0.260870,0.172932,0.650485,0.330769,0.136364
3,0.281195,0.202703,0.105372,0.288703,0.266628,0.395833,0.688679,1.000000,0.396932,0.240838,...,0.259961,0.180672,0.909091,0.404255,0.269841,0.573913,0.060150,0.398058,0.076923,0.079545
4,0.000000,0.364865,0.004132,0.000000,0.000000,0.043860,0.000000,0.000000,0.125599,0.041885,...,0.001547,0.037815,0.227273,0.042553,0.000000,0.000000,0.007519,0.038835,0.000000,0.000000
5,0.048858,0.081081,0.055785,0.104603,0.019665,0.000000,0.146226,0.144928,0.000000,0.149215,...,0.032495,0.021008,0.090909,0.095745,0.015873,0.373913,0.150376,0.203883,0.100000,0.000000


In [162]:
range(len(train) - look_back - 1)
a = train.iloc[0:3]
b = train.iloc[1:4]
dataX = []
dataX.append(a)
dataX.append(b)

In [164]:
# use the funcition to prepare the train and test datasets for modeling
trainX = create_dataset_X(train,look_back)
trainY = create_dataset_Y(train,look_back)

testX = create_dataset_X(test,look_back)
testY = create_dataset_Y(test,look_back)

# reshape input to be [samples , time steps , features]
trainX = np.array(trainX)
#trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))

(27, 3, 831)

In [ ]:
#debug

trainX.shape
#trainX

In [ ]:
# create and fit the LSTM network

model = Sequential()
model.add(LSTM(64, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# calculate the RMSE 
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))